# We are going to cover some of the most fundamental concepts of tensors using TensorFlow
## More specifically, we are going to cover:
* Introduction to tensors
* Getting information from Tensors
* Manipulating Tensors
* Tensors & NumPy
* USing @tf.function (a way to speed up regular python functions)
* Using GPUs with Tensorflow(or TPU)
* Exercises to try yourself.

# Tensorflow Docker instance command
nvidia-docker run -it -p 8888:8888 tensorflow_customized:latest-gpu-jupyter


In [1]:
!nvidia-smi

Mon Jul 11 02:38:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Check if GPU available
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

NameError: name 'tf' is not defined

# Introduction to Tensors

In [ ]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

In [ ]:
# Create a vector
vector = tf.constant([10,10])
vector

In [ ]:
# Check the dimensions of our vector
vector.ndim

In [ ]:
# Create a matrix (has more than one dimensions)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

In [ ]:
matrix.ndim

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with tensor
another_matrix

In [ ]:
# What's the number of dimensions of another matrix?
another_matrix.ndim

# The total number of dimensions is the number of elements in shape.

In [ ]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

In [ ]:
tensor.ndim

What we've created so far:
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.variable`

In [ ]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

In [ ]:
#Let's try change one of elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Let's try to change unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

🔑 **Note:** Rarely in practice will you need to decide wheer to use `tf.constant` or `tf.variable` to create tensors, as tensorflow dows this for you, if in doubt, use `tf.constant` and change it later if needed.

## Creating random tensors